In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 26.0 MB/s eta 0:00:00


In [22]:
from google.colab import files
import os

# Upload the zip file
uploaded = files.upload()

Saving Movement Summary_RA-10_Site_Marina Lundo.pdf to Movement Summary_RA-10_Site_Marina Lundo (1).pdf
Saving Movement Summary_RA-10_Site_Marina Lundo (2).pdf to Movement Summary_RA-10_Site_Marina Lundo (2).pdf
Saving Movement Summary_RA-9_Site_Marina Lundo.pdf to Movement Summary_RA-9_Site_Marina Lundo.pdf
Saving Movement Summary_RA-9_Site_Marina Lundo (2).pdf to Movement Summary_RA-9_Site_Marina Lundo (2).pdf
Saving Movement Summary_RA-8_Site_Marina Lundo.pdf to Movement Summary_RA-8_Site_Marina Lundo.pdf
Saving Movement Summary_RA-8_Site_Marina Lundo (2).pdf to Movement Summary_RA-8_Site_Marina Lundo (2).pdf
Saving Movement Summary_RA-7_Site_Marina Lundo.pdf to Movement Summary_RA-7_Site_Marina Lundo.pdf
Saving Movement Summary_RA-7_Site_Marina Lundo (2).pdf to Movement Summary_RA-7_Site_Marina Lundo (2).pdf
Saving Movement Summary_RA-6_Site_Marina Lundo.pdf to Movement Summary_RA-6_Site_Marina Lundo.pdf
Saving Movement Summary_RA-6_Site_Marina Lundo (2).pdf to Movement Summary_RA-

In [28]:
from collections import namedtuple
import pandas as pd
import pdfplumber
import re

In [29]:
# Dictionary to store extracted data
results = []
for filename in uploaded.keys():
    all_text = []
    with pdfplumber.open(filename) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:  # Ensure the page contains text
                all_text.append(f"Page {page_num}:\n{page_text}\n")

    # Join the text of all pages into a single string, separating by newlines
    full_text = "\n".join(all_text)
    data = []
    for line in full_text.split('\n'):
        data.append(line)
    pattern = r"Site: (\S+)\s\[(.*?)\s\(Site Folder: (.*?)\)\]"
    # Search the text using the regular expression
    match = re.search(pattern, data[2])
    if match:
        site_number = match.group(1)
        tmc_number = match.group(2)
        scenario = match.group(3)
    start_index = None
    data = full_text.split("\n")
    pattern = re.compile(r'Approach (\d+) [\d.]+ [\d.]+ [\d.]+ ([\d.]+) ([\d.]+) LOS (\w) ([\d.]+) ([\d.]+)')



    # Tracking the current approach name
    current_approach = None

    for line in data:
        if ":" in line:  # Detect approach name
            current_approach = line.split(":")[0].strip()
        elif pattern.match(line):  # Extract data using regex
            match = pattern.match(line)
            results.append({
                'Scenario': scenario,
                'Location': tmc_number,
                "Approach": current_approach,
                "Traffic Volume": int(match.group(1)),
                "Deg_Sat": float(match.group(2)),
                "Delay (sec)": float(match.group(3)),
                "LOS": match.group(4),
                "Queue Length (Veh)": float(match.group(5)),
                "Queue Length (m)": float(match.group(6))
            })
Output = pd.DataFrame(results)

In [30]:
Output.head(10)

,Scenario,Location,Approach,Traffic Volume,Deg_Sat,Delay (sec),LOS,Queue Length (Veh),Queue Length (m)
0,PM,RA-10,NorthEast,1408,0.643,10.6,B,4.3,30.5
1,PM,RA-10,NorthWest,397,0.970,67.8,F,10.6,74.5
2,PM,RA-10,SouthWest,711,0.419,9.3,A,1.8,12.7
3,AM,RA-10,NorthEast,1136,0.529,9.2,A,2.7,18.9
4,AM,RA-10,NorthWest,187,0.386,13.8,B,1.3,9.3
5,AM,RA-10,SouthWest,621,0.318,6.9,A,1.1,7.9
6,AM,RA-9,NorthEast,921,0.685,12.9,B,7.7,53.8
7,AM,RA-9,NorthWest,294,0.166,5.7,A,0.5,3.5
8,AM,RA-9,SouthWest,691,0.372,7.9,A,1.4,9.6
9,PM,RA-9,NorthEast,1177,1.026,44.9,E,42.1,294.8


In [31]:
Output.to_excel('Queue_Summary_Report.xlsx', index=False)